In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

sp500 = pd.read_csv("/Users/douglasallen/Library/CloudStorage/GoogleDrive-deallen1989@gmail.com/My Drive/Northeastern/Machine Learning/Final/Raw Data/SP500.csv")

Convert Date column to pandas datetime object, trim data to Jan 1 2022 and onward, and make chronological

In [2]:
sp500['DateTime'] = pd.to_datetime(sp500['Date'])
sp500['DateTime'] = np.where(sp500['DateTime'].dt.year > 2022, sp500['DateTime'] - pd.offsets.DateOffset(years=100), sp500['DateTime'])
# Anything after 2022 is changed to have 100 years subtracted because 2022 is the current year, change it as the years progress
sp500 = sp500.loc[(sp500['DateTime']>='2021-01-01')&(sp500['DateTime']<='2021-12-31')]
sp500 = sp500.loc[::-1].reset_index(drop=True)

sp500_close = sp500['Adj Close**']

dates = sp500['DateTime']



Pull in other data sets, do same transformation

In [3]:
unemp = pd.read_csv("/Users/douglasallen/Library/CloudStorage/GoogleDrive-deallen1989@gmail.com/My Drive/Northeastern/Machine Learning/Final/Raw Data/UNRATE.csv")
ten_year_rate = pd.read_csv("/Users/douglasallen/Library/CloudStorage/GoogleDrive-deallen1989@gmail.com/My Drive/Northeastern/Machine Learning/Final/Raw Data/DGS10.csv")
rec_exp = pd.read_csv("/Users/douglasallen/Library/CloudStorage/GoogleDrive-deallen1989@gmail.com/My Drive/Northeastern/Machine Learning/Final/Raw Data/Rec-Exp.csv")
cpi = pd.read_csv("/Users/douglasallen/Library/CloudStorage/GoogleDrive-deallen1989@gmail.com/My Drive/Northeastern/Machine Learning/Final/Raw Data/CPIAUCSL.csv")
vix = pd.read_csv("/Users/douglasallen/Library/CloudStorage/GoogleDrive-deallen1989@gmail.com/My Drive/Northeastern/Machine Learning/Final/Raw Data/VIX_History.csv")

unemp['DateTime'] = pd.to_datetime(unemp['DATE'])
ten_year_rate['DateTime'] = pd.to_datetime(ten_year_rate['DATE'])
rec_exp['DateTime'] = pd.to_datetime(rec_exp['DATE'])
cpi['DateTime'] = pd.to_datetime(cpi['DATE'])
vix['DateTime'] = pd.to_datetime(vix['DATE'])

print(unemp.tail(12))
print(ten_year_rate.head())
print(rec_exp.head())
print(cpi.head())
print(vix.head())


           DATE  UNRATE   DateTime
887  2021-12-01     3.9 2021-12-01
888  2022-01-01     4.0 2022-01-01
889  2022-02-01     3.8 2022-02-01
890  2022-03-01     3.6 2022-03-01
891  2022-04-01     3.6 2022-04-01
892  2022-05-01     3.6 2022-05-01
893  2022-06-01     3.6 2022-06-01
894  2022-07-01     3.5 2022-07-01
895  2022-08-01     3.7 2022-08-01
896  2022-09-01     3.5 2022-09-01
897  2022-10-01     3.7 2022-10-01
898  2022-11-01     3.7 2022-11-01
       DATE DGS10 ten_year   DateTime
0  1/2/1962  4.06     4.06 1962-01-02
1  1/3/1962  4.03     4.03 1962-01-03
2  1/4/1962  3.99     3.99 1962-01-04
3  1/5/1962  4.02     4.02 1962-01-05
4  1/8/1962  4.03     4.03 1962-01-08
         DATE  USREC   DateTime
0  1950-01-01      0 1950-01-01
1  1950-02-01      0 1950-02-01
2  1950-03-01      0 1950-03-01
3  1950-04-01      0 1950-04-01
4  1950-05-01      0 1950-05-01
         DATE  CPIAUCSL   DateTime
0  1947-01-01     21.48 1947-01-01
1  1947-02-01     21.62 1947-02-01
2  1947-03-01     22

Left merge with dates object above

In [4]:
unemp = pd.merge(dates.astype("datetime64[M]"),unemp,how='left',on = 'DateTime')
unemp = unemp['UNRATE']

cpi = pd.merge(dates.astype("datetime64[M]"),cpi,how='left',on = 'DateTime')
cpi = cpi['CPIAUCSL']

rec_exp = pd.merge(dates.astype("datetime64[M]"),rec_exp,how='left',on = 'DateTime')
rec_exp = rec_exp['USREC']

ten_year_rate = pd.merge(dates,ten_year_rate,how='left',on='DateTime')
ten_year_rate = ten_year_rate['ten_year']

vix = pd.merge(dates,vix,how='left',on='DateTime')
vix = vix['CLOSE']


Create vector dataframe

In [5]:
varma_data = pd.concat([sp500_close,unemp, cpi, ten_year_rate,vix],axis=1)
varma_data.rename(columns={'Adj Close**': 'sp500', 'UNRATE': 'unemp', 'CPIAUCSL': 'cpi',  'CLOSE': 'vix'}, inplace=True)

print(varma_data.head())

     sp500  unemp    cpi ten_year    vix
0  3700.65    6.4  262.2     0.93  26.97
1  3726.86    6.4  262.2     0.96  25.34
2  3748.14    6.4  262.2     1.04  25.07
3  3803.79    6.4  262.2     1.08  22.37
4  3824.68    6.4  262.2     1.13  21.56


In [6]:
from statsmodels.tsa.api import VAR

varma_model = VAR(np.asarray(varma_data,dtype = float))


In [7]:
results = varma_model.fit(5)

results.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Mon, 12, Dec, 2022
Time:                     01:02:58
--------------------------------------------------------------------
No. of Equations:         5.00000    BIC:                   -5.09614
Nobs:                     247.000    HQIC:                  -6.19955
Log likelihood:          -764.905    FPE:                0.000968971
AIC:                     -6.94319    Det(Omega_mle):     0.000587466
--------------------------------------------------------------------
Results for equation y1
           coefficient       std. error           t-stat            prob
------------------------------------------------------------------------
const     -1590.193079       629.931077           -2.524           0.012
L1.y1         0.968101         0.123090            7.865           0.000
L1.y2       -47.311709        48.829008           -0.969           0.333
L1.y3        -4.858633